In [ ]:
import numpy as np
import pandas as pd
import torch
from tc_python import *
from PIL import ImageColor
import os

In [19]:
os.environ['TC25A_HOME'] = '/Applications/Thermo-Calc-2025a.app/Contents/Resources/'
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk-19.jdk/Contents/Home'
os.environ['LSHOST'] = '128.194.43.55'
os.environ['LSERVRC'] = 'coe-vtls3.engr.tamu.edu'
os.environ['TCLM_LICENSE_FILE'] = '5093@128.194.43.55'

In [ ]:
def generate_quinary_compositions(increment):
    compositions = []
    steps = int(1 / increment) + 1
    small_value = 0

    for i in range(steps):
        for j in range(steps - i):
            for k in range(steps + 1 - i - j):
                for l in range(steps + 1 - i - j - k):
                    m = steps - 1 - i - j - k - l

                    # Convert steps to composition fractions
                    comp = [
                        i * increment,  # Au
                        j * increment,  # Ag
                        k * increment,  # Cu
                        l * increment,  # Pt
                        m * increment   # Al
                    ]

                    # Round to 2 decimal places
                    comp = [round(x, 2) for x in comp]

                    # Apply threshold constraint
                    if any(x < 0.02 for x in comp):
                        continue

                    compositions.append(comp)

    return compositions

In [ ]:
elements = ['Au', 'Ag', 'Cu', 'Pt', 'Al']
increment = 0.05 # Sampled 5%

# Generate compositions and convert to tensor
compositions = generate_quinary_compositions(increment)
compositions_tensor = torch.tensor(compositions, dtype=torch.double)
compositions_tensor

tensor([[0.0500, 0.0500, 0.0500, 0.0500, 0.8000],
        [0.0500, 0.0500, 0.0500, 0.1000, 0.7500],
        [0.0500, 0.0500, 0.0500, 0.1500, 0.7000],
        ...,
        [0.7500, 0.0500, 0.1000, 0.0500, 0.0500],
        [0.7500, 0.1000, 0.0500, 0.0500, 0.0500],
        [0.8000, 0.0500, 0.0500, 0.0500, 0.0500]], dtype=torch.float64)

In [22]:
len(compositions_tensor)

3876

In [23]:
x = compositions_tensor
results = []
with TCPython() as session:
    system = (
        session
        .select_database_and_elements("TCNOBL3", ["Au", "Ag", "Cu", "Pt", "Al"])
        .get_system()
        .with_property_model_calculation('Optical Properties - Noble')
        .set_temperature(298.15)
        .set_composition_unit(CompositionUnit.MOLE_FRACTION)
    )

    for xi in x:
        calc = (
            system
            .set_composition('Au', float(xi[0]))
            .set_composition('Ag', float(xi[1]))
            .set_composition('Cu', float(xi[2]))
            .set_composition('Pt', float(xi[3]))
        )

        result = calc.calculate()
        color_value = result.get_value_of('24-bit color (Opt. Prop. - Noble)')
        results.append([float(xi[0]), float(xi[1]), float(xi[2]), float(xi[3]), float(xi[4]), color_value])

02:16:34,603 [Thread-1] INFO  CalculationEngine: Starting Thermo-Calc 2025a, Java-version: 21.0.2

02:16:44,133 [Thread-1] INFO  CalculationEngine: Opening database::TCNOBL3::

02:16:45,325 [Thread-1] INFO  JavaWrapper: *** Invoking Gibbs Energy System v6 ***

02:16:51,916 [Thread-1] INFO  Py4JPropertyModelFactory: Creating Py4JPropertyModelFactory

02:16:59,873 [Thread-8] INFO  Py4JPropertyModel: 02:16:59,870 [Thread-1] INFO  CalculationEngine: Starting Thermo-Calc 2025a, Java-version: 21.0.2

02:17:01,463 [Thread-8] INFO  Py4JPropertyModel: 02:17:01,462 [Thread-1] INFO  SystemBuilder: Found matching previous system

02:17:03,539 [Thread-8] INFO  Py4JPropertyModel: 02:17:03,539 [Thread-1] INFO  JavaWrapper: *** Invoking Gibbs Energy System v6 ***

02:17:05,615 [Thread-8] INFO  Py4JPropertyModel: Running equilibrium at annealing temperature 298.15K...

02:17:09,017 [Thread-9] ERROR Py4JPropertyModel: Only 65 % of the phases have optical properties in current database, cannot compute co

In [24]:
df = pd.DataFrame(results, columns=["Au", "Ag", "Cu", "Pt", "Al", "24-bit color"])

In [ ]:
# Remove rows where the color_value is NaN
df = df.dropna(subset=[df.columns[-1]])

color = df.columns[-1]

# Function to convert a 24-bit color value to an RGB tuple
def convert_color_to_rgb(color_value):
    hex_code = hex(int(color_value))[2:].upper()
    rgb = ImageColor.getcolor("#" + hex_code, "RGB")
    return rgb

# Apply the function to the 24-bit color value to create (R, G, B)
rgb_tuples = df[color].apply(convert_color_to_rgb)

# Expand the tuple into three new columns: R, G, and B
df[['R', 'G', 'B']] = pd.DataFrame(rgb_tuples.tolist(), index=df.index)

df.to_csv('Data/color_quinary.csv', index=False)